In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os
import shutil
import matplotlib.pyplot as plt
from   torchvision import transforms
%matplotlib inline

##### 读取数据集

In [ ]:
base_dir = r'./data/FourWeather/'
train_dir= os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")

In [ ]:
train_transform = transforms.Compose([                                # 将图片在输入管道上做出一定变化，完成数据增强
    transforms.Resize(224),    
    transforms.RandomCrop(192),           
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(0.2),
    transforms.ColorJitter(brightness=0.5),
    transforms.ColorJitter(contrast=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
    
])

test_transform  = transforms.Compose([
    transforms.Resize((192,192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

In [ ]:
BATCH_SIZE = 8

In [ ]:
train_ds = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"train"),
    transform = train_transform
)

test_ds  = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"test"),
    transform = test_transform
)

train_dl = torch.utils.data.DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
test_dl  = torch.utils.data.DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=True)

In [ ]:
class_to_id = train_ds.class_to_idx
id_to_class = dict((v,k) for k,v in train_ds.class_to_idx.items())

##### 定义模型

In [ ]:
model = torchvision.models.vgg16(pretrained=True) 

In [ ]:
for p in model.features.parameters():
    p.requirs_grad = False                        # 设置卷积层里面的参数不再参与梯度下降与更新

In [ ]:
optim     = torch.optim.Adam(model.parameters(),lr=0.0001)
loss_fn   = nn.CrossEntropyLoss()
epoches   = 20
model.classifier[-1].out_features = 4             # 将输出分类数修改为目标数量 (本例为4)

In [ ]:
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

##### 定义fit函数

In [ ]:
def fit(model,trainloader,testloader,epoch):
    correct = 0       # 记录正确率
    total   = 0       # 记录训练样本总数量
    running_loss = 0
    
    model.train()     # 设置为训练模式 表示DropOut会起作用
    
    for x,y in trainloader:
        y_pred = model(x)
        loss   = loss_fn(y_pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)
            correct+=(y_pred == y).sum().item()
            total  += y.size(0)
            running_loss += loss.item()
    
    epoch_loss = running_loss / len(trainloader.dataset) # 得到每一个样本在本epoch下的平均loss
    epoch_acc  = correct / total # 得到整个样本的平均预测正确率
    
    test_correct      = 0        # 记录正确率
    test_total        = 0        # 记录训练样本总数量
    test_running_loss = 0
    
    model.eval()                 # 将模型修改为预测模式 表示DropOut失效 注意在验证数据集上不需要进行Dropout
    
    with torch.no_grad():
        for x,y in trainloader:
            y_pred = model(x)
            loss   = loss_fn(y_pred,y)
            y_pred = torch.argmax(y_pred,dim=1)
            test_correct+=(y_pred == y).sum().item()
            test_total  += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset) # 得到每一个样本在本epoch下的平均loss
    epoch_test_acc  = test_correct / test_total                   # 得到整个样本的平均预测正确率
    
    print(
            "epoch:","\t",epoch,"\t",
            "loss: ","\t",round(epoch_loss,3),"\t",
            "accuracy: ","\t",round(epoch_acc,3),"\t",
            "test_loss: ","\t",round(epoch_test_loss,3),"\t",
            "test_accuracy: ","\t",round(epoch_test_acc,3),"\t"
        )
    return epoch_loss,epoch_acc,epoch_test_loss,epoch_test_acc

##### 训练模型

In [ ]:
for epoch in range(epoches):
    epoch_loss,epoch_acc,epoch_test_loss,epoch_test_acc = fit(model,train_dl,test_dl,epoch)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

In [ ]:
plt.plot(range(1,epoches+1),train_loss,label = "train_loss")
plt.plot(range(1,epoches+1),test_loss,label = "test_loss")
plt.legend()

In [ ]:
plt.plot(range(1, epoches+1), train_acc, label='train_acc')
plt.plot(range(1, epoches+1), test_acc, label='test_acc')
plt.legend()